In [1]:
import pandas as pd
import os
import sys
import ast

# Import library with current code functions
sys.path.append(os.path.join("..", "lib"))
import general_functions as gf, files_paths as fp

In [2]:
MAIN_LIST_LABELER = gf.find_files_in_all_subdirectories([fp.DATASET_LOCAL, fp.DATASET_YT], fp.VD_LABELED_L0)

In [3]:
gf.find_files_in_all_subdirectories([os.path.join('.', 'result_sequences')], '*.CSV')

['.\\result_sequences\\DATASET_SEQUENCES_1.CSV',
 '.\\result_sequences\\DATASET_SEQUENCES_2.CSV',
 '.\\result_sequences\\DATASET_SEQUENCES_3.CSV',
 '.\\result_sequences\\DATASET_SEQUENCES_4.CSV',
 '.\\result_sequences\\DATASET_SEQUENCES_5.CSV',
 '.\\result_sequences\\DATASET_SEQUENCES_6.CSV',
 '.\\result_sequences\\DATASET_SEQUENCES_7.CSV']

In [4]:
def parse_string_to_object(label_measure_string):
    parsed_object = ast.literal_eval(label_measure_string) 
    return parsed_object

In [5]:
def get_emotion_with_min_value(parsed_object):
    emotion_with_min_value = min(parsed_object.items(), key=lambda x: x[1][1])
    emotion_name, emotion_data = emotion_with_min_value
    return emotion_name, emotion_data

In [6]:
num_file_sequences = len(gf.find_files_in_all_subdirectories([os.path.join('.', 'result_sequences')], '*.CSV'))
OUTPUT_NAME = os.path.join('result_sequences','DATASET_SEQUENCES_' + str(num_file_sequences+1) + '.CSV')
DATASET_SEQUENCES = pd.DataFrame()

idx_sample = 0
for i, current_path_location in enumerate(MAIN_LIST_LABELER[:]):
    VD_LABELED_DT = pd.read_csv(current_path_location)
    VD_LABELED_DT.drop(columns=['Unnamed: 0'], inplace=True)

    print(f"Current video ({i+1} of {len(MAIN_LIST_LABELER)}):", current_path_location)

    for group in VD_LABELED_DT.groupby('label_measures'):
        
        if len(group[0]) > 5:
            label_measure_string = group[0]
            parsed_object = parse_string_to_object(label_measure_string)
            emotion_name, emotion_data = get_emotion_with_min_value(parsed_object)

            emotion_euclidean_dist = emotion_data[1]
            emotion_reference_seed = emotion_data[2]

            current_insertion = group[1].copy()
            current_insertion['label'] = str(emotion_name)
            current_insertion['euclidean_dist'] = emotion_euclidean_dist
            current_insertion['reference_seed'] = emotion_reference_seed
            current_insertion['sample_id'] = idx_sample

            cols = list(current_insertion.columns)
            cols.insert(1, cols.pop(cols.index('sample_id')))
            current_insertion = current_insertion[cols]

            idx_sample += 1

            DATASET_SEQUENCES = pd.concat([DATASET_SEQUENCES, current_insertion], axis=0)

print(f'\nSaving file {OUTPUT_NAME}...')
DATASET_SEQUENCES.to_csv(OUTPUT_NAME)

Current video (1 of 70): ..\Dataset\DD-Local\VD_D_0000000043\VD_LABELED_L0.CSV
Current video (2 of 70): ..\Dataset\DD-Local\VD_D_0000000044\VD_LABELED_L0.CSV
Current video (3 of 70): ..\Dataset\DD-Local\VD_D_0000000045\VD_LABELED_L0.CSV
Current video (4 of 70): ..\Dataset\DD-Local\VD_D_0000000046\VD_LABELED_L0.CSV
Current video (5 of 70): ..\Dataset\DD-Local\VD_D_0000000047\VD_LABELED_L0.CSV
Current video (6 of 70): ..\Dataset\DD-Local\VD_D_0000000048\VD_LABELED_L0.CSV
Current video (7 of 70): ..\Dataset\DD-Local\VD_D_0000000049\VD_LABELED_L0.CSV
Current video (8 of 70): ..\Dataset\DD-Local\VD_D_0000000050\VD_LABELED_L0.CSV
Current video (9 of 70): ..\Dataset\DD-Local\VD_D_0000000051\VD_LABELED_L0.CSV
Current video (10 of 70): ..\Dataset\DD-Local\VD_D_0000000052\VD_LABELED_L0.CSV
Current video (11 of 70): ..\Dataset\DD-Local\VD_D_0000000053\VD_LABELED_L0.CSV
Current video (12 of 70): ..\Dataset\DD-Local\VD_D_0000000054\VD_LABELED_L0.CSV
Current video (13 of 70): ..\Dataset\DD-Local\VD_